
#### 10.2 단어 자동 완성

- 영문자 4개로 구성된 단어를 학습시켜서, 3글자만 주어지면 나머지 한 글자를 추천하는 프로그램

**!** dynamic_rnn의 sequence_length 옵션을 사용하면 가변 길이의 단어를 학습시킬 수 있다. 
짧은 단어는 가장 긴 단어의 길이 만큼 뒷부분을 0으로 채우고, 해당 단어의 길이를 계산해서 sequence_length로 넘겨준다 (batch_size만큼의 배열로)

- 학습시킬 데이터는 영문자로 구성된 임의의 단어.
- 한 글자를 하나의 단계로 본다. (한 글자 = 한 단계 입력값 / 총 글자 = 전체 단계)

![자동완성 모델](\img\img3.JPG)



In [1]:
import tensorflow as tf
import numpy as np

# 각 글자에 해당하는 인덱스를 one-hot 코딩으로 표현한다.
# 이를 위해 알파벳 글자들을 배열에 넣고
char_arr = ['a', 'b','c','d','e'
            ,'f','g','h','i','j'
            ,'k','l','m','n','o','p'
            ,'q','r','s','t','u'
            ,'v','w','x','y','z']
# {'a':0, 'b':1, 'c':2,...'j':9...}

# 해당 글자의 인덱스를 구할 수 있는 연관 배열(딕셔너리)를 만든다.
num_dic = {n: i for i, n in enumerate(char_arr)}
dic_len = len(num_dic)


# 학습에 사용할 단어를 배열로 저장한다.
seq_data = ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load',
           'love', 'kiss', 'kind']



-----------

단어들을 학습에 사용할 수 있는 형식으로 변환하는 함수 작성.  
1. 입력값 용 : 단어의 처음 세 글자의 알파벳 인덱스를 구한 배열을 만든다.
    - input = [num_dic[n] for n in seq[:-1]]
2. 출력값 용 : 마지막 글자의 알파벳 인덱스를 구한다.
    - target = num_dic[seq[-1]]
3. 입력값을 원-핫 인코딩으로 변환
    - iput_batch.append(np.eye(dic_len)[input])

```
! ) 예
데이터 : deep  
-> 입력값 : d, e, e  
-> 인덱스를 구해 배열로 : [3, 4, 4]  
-> 원-핫 코딩 : [[0, 0, 0, 1, 0, ...,0]  
                [0, 0, 0, 0, 1, ...,0]  
                [0, 0, 0, 0, 1, ...,0]]  
   ```             
실측값 : p 의 인덱스 15, 원-핫 인코딩 하지 않음.
    -> 손실함수로 sparse_softmax_cross_entropy_with_logits를 사용하기 때문
    **!** 위 함수는 실측값(labels) 값에 원-핫 인코딩을 사용하지 않아도 자동으로 변환하여 계산한다.  
    

In [6]:
def make_batch(seq_data):
    input_batch = []
    target_batch = []
    
    for seq in seq_data:
        input = [num_dic[n] for n in seq[:-1]]
        target = num_dic[seq[-1]]
        input_batch.append(np.eye(dic_len)[input])
        target_batch.append(target)
    
    return input_batch, target_batch
        

In [3]:
# 데이터 전처리 끝남.

#######
# 신경망 모델 구성
##########

learning_rate = 0.01
n_hidden = 128
total_epoch = 30
n_step = 3    # 단어중 처음 3글자를 단계적으로 학습. 
# 입력, 출력값은 원-핫 인코딩을 사용하기 때문에 알파벳 배열 크기와 같다.
# sparse_softmax 사용시 실측값:인덱스 숫자, 예측모델 출력값:원-핫 인코딩
n_input = n_class = dic_len 


X = tf.placeholder(tf.float32, [None, n_step, n_input])
# 실측값, batch_size에 해당하는 하나의 차원만 있다. (원-핫 코딩이 아니고 인덱스 그대로)
Y = tf.placeholder(tf.int32, [None])
W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))



In [4]:
# 2개의 RNN 셀 생성
# 여러 셀을 조합해서 심층 신경망을 만든다.
cell1 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
# 드롭아웃 기법
cell1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob=0.5)
cell2 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)


# 샐성한 셀들을 MultiRNNCell 함수로 조합하고
multi_cell = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2])
# dynamic_rnn 함수로 심층 순환 신경망 (Deep RNN)을 만든다.
outputs, states = tf.nn.dynamic_rnn(multi_cell, X, dtype=tf.float32)

# 최종 출력층을 만든다.
outputs = tf.transpose(outputs, [1, 0, 2])
outputs = outputs[-1]
model = tf.matmul(outputs, W) + b

# 손실함수 
cost = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits = model, labels = Y))
# 최적화 함수
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)


In [7]:


###########
# 신경망 학습
######

sess = tf.Session()
sess.run(tf.global_variables_initializer())

# seq_data에 저장한 단어들을 입력값(처음 세글자)와 실측값(마지막 글자)로 분리
input_batch, target_batch = make_batch(seq_data)

for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost],
                      feed_dict = {X: input_batch, Y: target_batch})
    print('Epoch::','%04d' % (epoch + 1),
         'cost = ','{:.6f}'.format(loss))

print('최적화 완료!')



# 예측단어와 정확도 출력
# Y가 정수이기 때문에 agrmac로 변환한 예측값도 정수로 바꿈.
prediction = tf.cast(tf.argmax(model, 1), tf.int32)
prediction_check = tf.equal(prediction, Y)
accuracy = tf.reduce_mean(tf.cast(prediction_check, tf.float32))

# 예측 모델 실행
input_batch, target_batch = make_batch(seq_data)
predict, accuracy_val = sess.run([prediction, accuracy],
                                feed_dict = {X: input_batch, Y: target_batch})


Epoch:: 0001 cost =  3.523435
Epoch:: 0002 cost =  2.600816
Epoch:: 0003 cost =  1.452710
Epoch:: 0004 cost =  1.224106
Epoch:: 0005 cost =  0.537529
Epoch:: 0006 cost =  0.550484
Epoch:: 0007 cost =  0.520311
Epoch:: 0008 cost =  0.455570
Epoch:: 0009 cost =  0.322111
Epoch:: 0010 cost =  0.339305
Epoch:: 0011 cost =  0.281957
Epoch:: 0012 cost =  0.134980
Epoch:: 0013 cost =  0.038312
Epoch:: 0014 cost =  0.219446
Epoch:: 0015 cost =  0.347925
Epoch:: 0016 cost =  0.171057
Epoch:: 0017 cost =  0.059178
Epoch:: 0018 cost =  0.031553
Epoch:: 0019 cost =  0.141151
Epoch:: 0020 cost =  0.186091
Epoch:: 0021 cost =  0.076618
Epoch:: 0022 cost =  0.011051
Epoch:: 0023 cost =  0.017619
Epoch:: 0024 cost =  0.018152
Epoch:: 0025 cost =  0.039708
Epoch:: 0026 cost =  0.014823
Epoch:: 0027 cost =  0.015523
Epoch:: 0028 cost =  0.031864
Epoch:: 0029 cost =  0.130121
Epoch:: 0030 cost =  0.035207
최적화 완료!


In [8]:
# 모델이 예측한 값들을 가지고, 각각 값에 해당하는 인덱스의 알파벳을 가져와 예상 단어 출력

predict_words = []
for idx, val in enumerate(seq_data):
    last_char = char_arr[predict[idx]]
    predict_words.append(val[:3] + last_char)
    
print("=======예측 결과=======")
print("입력값 : ",[w[:3] + ' ' for w in seq_data])
print("예측값 : ", predict_words)
print('정확도 : ', accuracy_val)

=======예측 결과=======
입력값 :  ['wor ', 'woo ', 'dee ', 'div ', 'col ', 'coo ', 'loa ', 'lov ', 'kis ', 'kin ']
예측값 :  ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load', 'love', 'kiss', 'kind']
정확도 :  1.0
